In [1]:
from SpiderV2 import Spider
from IndexV2 import Database, Index, RawInfoIndex
from Tokenize import Tokenize
from Searcher import Search
import time

In [2]:
db = Database(SearchEngine="Demo2")
index = Index()
raw_storage = RawInfoIndex()
tokenizer = Tokenize()
raw_storage.read_from_database(db)
index.read_fw_index_from_database(db)
index.read_ivi_index_from_database(db)

In [1]:

url = "https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki"
#"https://en.wikipedia.org/wiki/Climate_change", "https://riskofrain.fandom.com/wiki/Main_Page", "https://www.thairath.co.th/news/local/bangkok/2635906" 
Spider.set_base_domains(["https://www.thairath.co.th/", "https://riskofrain.fandom.com/wiki/Main_Page", "https://en.wikipedia.org/", "https://iot-kmutnb.github.io/", "https://riskofrain2.fandom.com/"])
Spider.max_depth = 0

spider = Spider(url)
success = spider.generate_next_soup()

raw_storage.read_from_database(db)

if success:
    info = spider.crawl({})
    raw_storage.modify_index(Spider.queue[Spider.queue_front-1], info[0], info[1], info[2])
    raw_storage.save_to_database(db)
else:
    if Spider.get_base_domain(url) in Spider.all_robots:
        Spider.unaccessible_urls.append(url)
    print("Cannot crawl to the website.")

NameError: name 'Spider' is not defined

In [3]:
for url in raw_storage.index:
    tokens = tokenizer.tokenize(raw_storage.index[url]["text"])
    tokens = tokenizer.filter(tokens)
    tokens = tokenizer.make_counter(tokens)
    index.modify_index(url, tokens)
raw_storage.save_to_database(db)
index.save_fw_index_to_database(db)
index.save_ivi_index_to_database(db)

In [5]:
print(raw_storage.get_ref_count("https://riskofrain.fandom.com/wiki/Main_Page"))

1


In [7]:
raw_storage.remove_urls(["https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki"])
index.remove_urls(raw_storage.url_to_be_deleted)

In [10]:
print(index.ivi_index["risk"])

{'https://en.wikipedia.org/wiki/Climate_change': 39, 'https://riskofrain.fandom.com/wiki/Main_Page': 26, 'https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki': 18}


In [4]:
print(Search().search(["risk","of","rain"], raw_storage.index, index.fw_index, index.ivi_index))

0.03174749011659252
0.24965436127615284
0.19964588775409575
[('https://riskofrain.fandom.com/wiki/Main_Page', 0.26965436127615283, 1, {}), ('https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki', 0.21964588775409574, 1, {}), ('https://en.wikipedia.org/wiki/Climate_change', 0.03174749011659252, 0, {})]
